In [3]:
import graph_characteristics_lib as gc

In [5]:
names = [ ["dnc","dnc","crimson","d"] , ["fb-forum", "fb", "green","d"], ["talk_eo", "eo" ,"olive","d"],
          ["email-eu3", "eu3", "yellow","d"], ["email-eu2", "eu2", "magenta","d"], 
          ["racoon", "rac", "brown","u"], ["primate","pri",  "blue","u"],  ["workplace_2013", "wp", "black","u"],
           ["ht09", "ht","red","u"], ["weaver", "wea",  "pink","u"] ]

In [4]:
def find_sep(x):
    for s in x:
        if s not in ['0','1','2','3','4','5','6','7','8','9']:
            return s
    return None

def read_graph(path,s, dire):
    g = set()
    f = open(path+s+".csv", "r")
    sep = find_sep(f.readline())
    f.close()
    f = open(path+s+".csv", "r")
    for x in f:
        x = x[:-1]
        r = x.split(sep)
        u,v,t = r[0],r[1],r[2]
        if v != u:
            t = float(t)
            g.add( (u,v,int(t)) )
            if dire == "u":
                g.add( (v,u,int(t)) )
    return g

def write_graph(path, s, g):
    f = open(path+s+".csv", "w")
    for x in g:
        f.write( str(x) )
    f.close()

In [14]:
fold_d = "datasets/networks/"
for i in range(len(names)):
    cur = fold_d + names[i][0]+".csv"
    g = read_graph(fold_d,names[i][0], names[i][-1])
#     print(names[i][0], nb_nodes(cur),nb_time_events(cur),nb_edges(cur))
    count = (names[i][-1],len(gc.nodes(g)))
#     count = nb_time_events(cur)
    names[i].insert(0,count)
print(names)
names.sort(reverse = True)
names = [ names[i][1:] for i in range(len(names))  ]
print(names)
for i in range(len(names)):
    g = read_graph(fold_d,names[i][0], names[i][-1])
    print(names[i][0], "& ",names[i][1], "&" ,names[i][-1],   "& $", len(gc.nodes(g)), "$ & $", len(gc.events(g)), "$ & $" ,len(g), "$ \\\\")

[[('u', 445), 'weaver', 'wea', 'pink', 'u'], [('u', 113), 'ht09', 'ht', 'red', 'u'], [('u', 92), 'workplace_2013', 'wp', 'black', 'u'], [('u', 25), 'primate', 'pri', 'blue', 'u'], [('u', 24), 'racoon', 'rac', 'brown', 'u'], [('d', 7288), 'talk_eo', 'eo', 'olive', 'd'], [('d', 1866), 'dnc', 'dnc', 'crimson', 'd'], [('d', 899), 'fb-forum', 'fb', 'green', 'd'], [('d', 162), 'email-eu2', 'eu2', 'magenta', 'd'], [('d', 89), 'email-eu3', 'eu3', 'yellow', 'd']]
[['weaver', 'wea', 'pink', 'u'], ['ht09', 'ht', 'red', 'u'], ['workplace_2013', 'wp', 'black', 'u'], ['primate', 'pri', 'blue', 'u'], ['racoon', 'rac', 'brown', 'u'], ['talk_eo', 'eo', 'olive', 'd'], ['dnc', 'dnc', 'crimson', 'd'], ['fb-forum', 'fb', 'green', 'd'], ['email-eu2', 'eu2', 'magenta', 'd'], ['email-eu3', 'eu3', 'yellow', 'd']]
weaver &  wea & u & $ 445 $ & $ 23 $ & $ 2666 $ \\
ht09 &  ht & u & $ 113 $ & $ 5246 $ & $ 41636 $ \\
workplace_2013 &  wp & u & $ 92 $ & $ 7104 $ & $ 19654 $ \\
primate &  pri & u & $ 25 $ & $ 19 $ &

In [58]:
def find_names(x):
    s = ""
    for e in x:
        if (ord(e) >= ord('a') and ord('z') >= ord(e)) or (ord(e) >= ord('A') and ord('Z') >= ord(e)) or e == '-' or e == '_':
            s += e
    return s
        
def read_next_number(x,i):
    print("start", (x,i))
    num = ""
    while (i < len(x)) and (not ((ord(x[i]) >= ord('0') and ord('9') >= ord(x[i]) ))):
        i += 1
    print("sort")
    if i >= len(x):
        print("ixi")
        return False
    
    #first part
    while (ord(x[i]) >= ord('0') and ord('9') >= ord(x[i]) ):
        num += x[i]
        i += 1
    print("firtst", num)
    if (x[i] != '.'):
        print("ret")
        return float(num), i+1
    else:
        num += '.'
        i += 1
        
    #second part
    while (ord(x[i]) >= ord('0') and ord('9') >= ord(x[i]) ):
        num += x[i]
        i += 1
    return float(num), i+1
    
def add_line(x):
    l = []
    nb,ind = read_next_number(x,0)
    l.append(nb)
    while True:
        res = read_next_number(x,ind)
        if type(res) != bool:
            nb,ind = res
            l.append(nb)
        else:
            break
    return l
    
def read_tex(s):
    f = open(s,"r")
    d = dict()
    i = 0
    for x in f:
        av = []
        dev = []
        x = x[:-1]
        if i%2 == 0:
            nam = find_names(x)
            print("name", nam)
        d[nam] = dict()
        if i%2 == 0:
            d[nam]["av"] = add_line(x)  
            print("*******", d[nam]["av"])
        else:
            d[nam]["dev"] = add_line(x)
            print("*******", d[nam]["dev"])
        i += 1
    f.close()
    return d

In [60]:
d = read_tex("table_charac__clus.tex")

name dnc
start ('dnc & $ 716.9 $, & $ 477.2 $, & $ 434.4 $, & $ 1.197 $, & $ 0.353 $,\\\\ ', 0)
sort
firtst 716
start ('dnc & $ 716.9 $, & $ 477.2 $, & $ 434.4 $, & $ 1.197 $, & $ 0.353 $,\\\\ ', 14)
sort
firtst 477
start ('dnc & $ 716.9 $, & $ 477.2 $, & $ 434.4 $, & $ 1.197 $, & $ 0.353 $,\\\\ ', 27)
sort
firtst 434
start ('dnc & $ 716.9 $, & $ 477.2 $, & $ 434.4 $, & $ 1.197 $, & $ 0.353 $,\\\\ ', 40)
sort
firtst 1
start ('dnc & $ 716.9 $, & $ 477.2 $, & $ 434.4 $, & $ 1.197 $, & $ 0.353 $,\\\\ ', 53)
sort
firtst 0
start ('dnc & $ 716.9 $, & $ 477.2 $, & $ 434.4 $, & $ 1.197 $, & $ 0.353 $,\\\\ ', 66)
sort
ixi
******* [716.9, 477.2, 434.4, 1.197, 0.353]
start (' & $ 0 $, & $ 0.408 $, & $ 0.674 $, & $ 1.158 $, & $ 0.499 $,\\\\ ', 0)
sort
firtst 0
ret
start (' & $ 0 $, & $ 0.408 $, & $ 0.674 $, & $ 1.158 $, & $ 0.499 $,\\\\ ', 7)
sort
firtst 0
start (' & $ 0 $, & $ 0.408 $, & $ 0.674 $, & $ 1.158 $, & $ 0.499 $,\\\\ ', 20)
sort
firtst 0
start (' & $ 0 $, & $ 0.408 $, & $ 0.674 $, & $ 

In [61]:
d

{'dnc': {'dev': [0.0, 0.408, 0.674, 1.158, 0.499]},
 'fb': {'dev': [0.0, 4.949, 5.656, 1.414, 0.707]},
 'eo': {'dev': [0.0, 6.734, 2.66, 1.138, 0.707]},
 'eu': {'dev': [0.0, 0.32, 16.3, 10.87, 0.414]},
 'rac': {'dev': [0.0, 0.0, 1.029, 6.447, 16.57]},
 'pri': {'dev': [0.0, 0.0, 0.73, 4.848, 6.571]},
 'wp': {'dev': [0.0, 0.0, 0.0, 26.26, 5.863]},
 'ht': {'dev': [0.0, 0.0, 0.146, 45.92, 9.386]},
 'wea': {'dev': [0.0, 0.0, 0.0, 0.0, 0.0]}}

In [62]:
d = read_tex("table_charac__tri.tex")

name dncfbeoeueuracpriwphtwea
start ('dnc & $ 67891 $, & $ 67371 $, & $ 65513 $, & $ 1437. $, & $ 1789. $, & $ 0 $, & $ 18340 $, & $ 55427 $, & $ 54.44 $, & $ 159.8 $,fb & $ 44409 $, & $ 43665 $, & $ 36664 $, & $ 16869 $, & $ 17317 $, & $ 0 $, & $ 62.93 $, & $ 523.9 $, & $ 941.8 $, & $ 490.4 $,eo & $ 36777 $, & $ 35758 $, & $ 35449 $, & $ 38.5 $, & $ 41.66 $, & $ 0 $, & $ 10081 $, & $ 63102 $, & $ 0.707 $, & $ 13.19 $,eu3 & $ 71661 $, & $ 71065 $, & $ 62808 $, & $ 75308 $, & $ 80577 $, & $ 0 $, & $ 18807 $, & $ 67764 $, & $ 26614 $, & $ 5983. $,eu2 & $ 15248 $, & $ 15203 $, & $ 14135 $, & $ 67072 $, & $ 72461 $, & $ 0 $, & $ 33373 $, & $ 10672 $, & $ 10156 $, & $ 29594 $,rac & $ 28737 $, & $ 28737 $, & $ 28728 $, & $ 28648 $, & $ 27852 $, & $ 0 $, & $ 0.0 $, & $ 1063. $, & $ 14171 $, & $ 14952 $,pri & $ 74325 $, & $ 74315 $, & $ 74334 $, & $ 73228 $, & $ 70844 $, & $ 0 $, & $ 127.2 $, & $ 57.98 $, & $ 711.3 $, & $ 3824. $,wp & $ 46805 $, & $ 46805 $, & $ 46805 $, & $ 27557 $, & $ 18512

In [63]:
d

{'dncfbeoeueuracpriwphtwea': {'av': [67891.0,
   67371.0,
   65513.0,
   1437.0,
   1789.0,
   0.0,
   18340.0,
   55427.0,
   54.44,
   159.8,
   44409.0,
   43665.0,
   36664.0,
   16869.0,
   17317.0,
   0.0,
   62.93,
   523.9,
   941.8,
   490.4,
   36777.0,
   35758.0,
   35449.0,
   38.5,
   41.66,
   0.0,
   10081.0,
   63102.0,
   0.707,
   13.19,
   3.0,
   71661.0,
   71065.0,
   62808.0,
   75308.0,
   80577.0,
   0.0,
   18807.0,
   67764.0,
   26614.0,
   5983.0,
   2.0,
   15248.0,
   15203.0,
   14135.0,
   67072.0,
   72461.0,
   0.0,
   33373.0,
   10672.0,
   10156.0,
   29594.0,
   28737.0,
   28737.0,
   28728.0,
   28648.0,
   27852.0,
   0.0,
   0.0,
   1063.0,
   14171.0,
   14952.0,
   74325.0,
   74315.0,
   74334.0,
   73228.0,
   70844.0,
   0.0,
   127.2,
   57.98,
   711.3,
   3824.0,
   46805.0,
   46805.0,
   46805.0,
   27557.0,
   18512.0,
   0.0,
   0.0,
   0.0,
   61321.0,
   12189.0,
   18634.0,
   18635.0,
   18634.0,
   92605.0,
   89851.0,
   0.0